We try to create a class for dictionary and set.

In [1]:
#################
# class for element
class Assoc:
    def __init__(self,key,value):
        self.key = key
        self.value = value

    def __lt__(self,other):
        return self.key < other.key
    
    def __le__(self,other):
        return self.key <= other.key
    
    def __str__(self):
        return 'Assoc({0},{1})'.format(self.key,self.value)
    
######################
# class for dictionary
class DictList:
    def __init__(self):
        self._elems = []
        
    def is_empty(self):
        return self._elems == []
    
#####################
# bi-section search for ordered dictionary
def bisearch(lis,key):
    low, high = 0,len(lis)-1
    while low <= high:
        mid = (low+high)//2
        if key == lis[mid].key:
            return lis[mid].value
        if key<lis[mid].key:
            high= mid-1
        else:
            low = mid+1
            

We try to create dictionary by hash table.

In [2]:
# one sample hash function
def str_hash(s):
    h1 = 0
    for c in s:
        h1 = h1*29 + ord(c)
    return h1

Now we gonna focus on binary sorted tree.

In [3]:
def bt_search(btree,key):
    bt = btree
    while bt is not None:
        entry = bt.data
        if key < entry.key:
            bt = bt.left
        elif key > entry.key:
            bt = bt.right
        else:
            return entry,value
    return None
#############
# Following is BinTNode defined in char6
class BinTNode:
    def __init__(self,dat,left=None,right=None):
        self.data = dat
        self.left = left
        self.right = right
#############

class DictBinTree:
    def __init__(self):
        self._root = None
        
    def is_empty(self):
        return self._root == None
    
    def search(self,key):
        bt = self._root
        while bt is not None:
            entry = bt.data
            if key < entry.key:
                bt = bt.left
            elif key > entry.key:
                bt = bt.right
            else:
                return entry,value
        return None
    
    def insert(self,key,value):
        bt = self._root
        if bt == None:
            self._root = BinTNode(Assoc(key,value))
            return 
        while bt!= None:
            if key < bt.key:
                if bt.left == None:
                    bt.left = BinTNode(Assoc(key,value))
                    return 
                bt = bt.left
            elif key > bt.key:
                if bt.right == None:
                    bt.right = BinTNode(Assoc(key,value))
                    return 
                bt = bt.right
            else:
                bt.value = value
        return 
            
    def entries(self):
        t,s = self._root,SStack()
        while t!=None or not s.is_empty():
            while t != None:
                s.push(t)
                t = t.left
            t = s.pop()
            yield t.data.key, t.data.value
            t = t.right
            
    def delete(self,key):
        p, q = None, self._root
        while q is not None and q.data.key != key:
            p = q
            if key < p.key:
                q = p.left
            else:
                q = q.right
            if q is None:
                return 
            
        if self._root.key == key:
            self._root = None
            return 
        
        # then we gonna find the leftest node of right son tree and use it to substitute the deleted node
        t1,t2 = q,q.right
        if t2 == None:
            q = q.left
            return 
        if t2.left == None:
            q.right = t2.right
            q.key = t2.key
            q.value = t2.value
        while t2.left != None:
            t1 = t2
            t2 = t2.left
        q.key = t2.key
        q.value = t2.value
        t1.left = t2.right
        return 
    
    def print(self):
        for k,v in self.entries():
            print(k,v)
            
def build_BT(entries):
    bt = DictBinTree()
    for k,v in entries:
        bt.insert(k,v)

To minimize the average search time, this part would try to optimize the binary tree.

In [4]:
class DictOptBinTree(DictBinTree):
    def __init__(self,seq):
        DictBinTree.__init__(self)
        data = sorted(seq)
        self._root = DictOptBinTree.buildOBT(data,0,len(data)-1)
        
    @staticmethod
    def buildOBT(data,start,end):
        if start > end:
            return None
        mid = (start+end)//2
        left = DictOptBinTree.buildOBT(data,start,mid-1)
        right = DictOptBinTree.buildOBT(data,mid+1,end)
        return BinTNode(Assoc(*data[mid]),left,right)

If weights on each result is given, then we need a more general method to create the optimal tree.

In [5]:
def build_opt_btree(wp,wq):
    num = len(wp)+1
    if len(wq)!=num:
        raise ValueError('Arguments error')
    w = [[0]*num for i in range(num)] # stores the sum of weights from i to j,including internal and external
    c = [[0]*num for i in range(num)] # stores the price of tree(i,j)
    r = [[0]*num for i in range(num)] # stores the optimal root of tree(i,j)
    
    for i in range(num):
        w[i][i] = wq[i]
        for j in range(i+1,num):
            w[i][j] = w[i][j-1] + wp[j-1] + wq[j]
    for i in range(0,num-1):
        c[i][i+1] = w[i][i++1]
        r[i][i+1] = i
        
    for m in range(2,num):
        for i in range(0,num-m):
            k0,j = i,i*m
            wmin = inf
            for k in range(i,j):
                if c[i][k] + c[k+1][j]<wmin:
                    mwin = c[i][k] + c[k+1][j]
                    k0 = k
            c[i][j] = w[i][j] + wmin
            r[i][j] = k0
    return c,r

Balance binary sort tree(AVL)

In [6]:
class AVLNode(BinTNode):
    def __init__(self,data):
        BinTNode.__init__(self,data)
        self.bf = 0
        
class DictAVL(DictBinTree):
    def __init__(self):
        DictBinTree.__init__(self)
    ###############
    # We consider how to insert and delete.
    @staticmethod
    def LL(a,b):
        a.left = b.right
        b.right = a
        a.bf = b.bf = 0
        return b
    @staticmethod
    def RR(a,b):
        a.right = b.left
        b.left = a
        a.bf = b.bf
        return b
    @staticmethod
    def LR(a,b):
        c = b.right
        b.right = c.left
        a.left = c.right
        c.left,c.right = b,a
        if c.bf == 0:
            a.bf = b.bf = 0
        elif c.bf == 1:
            b.bf,a.bf = 0,-1
        else:
            b.bf,a.bf = 1,0
        c.bf = 0
        return c
    
    @staticmethod
    def RL(a,b):
        c = b.left
        b.left = c.right
        a.right = c.left
        c.right,c.left = b,a
        if c.bf == 0:
            a.bf = b.bf = 0
        elif c.bf == 1:
            a.bf,b.bf = 0,-1
        else:
            a.bf,b.bf = 1,0
        c.bf = 0
        return c
    
    def insert(self,key,value):
        a = p = self._root
        if a == None:
            self._root = AVLNode(Assoc(key,value))
            return
        pa = q = None
        while p != None:
            if key == p.data.key:
                p.data.value = value
                return 
            if p.bf != 0:
                pa,a = q,p
            q = p
            if key < p.data.key:
                p = p.left
            else:
                p = p.right
    
        node = AVLNode(Assoc(key,value))
        if key < q.data.key:
            q.left = node
        else:
            q.right = node
        if key < a.data.key:
            p = b = a.left
            d = 1
        else:
            p = b = a.right
            d = -1

        while p != None:
            if key < p.data.key:
                p.bf = 1
                p = p.left
            else:
                p.bf = -1
                p = p.right
        if a.bf == 0:
            a.bf = d
            return 
        if a.bf == -d:
            a.bf = 0
            return 
        
        if d == 1:
            if b.bf == 1:
                b = DictAVL.LL(a,b)
            else:
                b = DictAVL.LR(a,b)
        else:
            if b.bf == 1:
                b = DictAVL.RL(a,b)
            else:
                b = DictAVL.RR(a,b) 
        if pa == None:
            self._root = b
        else:
            if pa.left == a:
                pa.left = b
            else:
                pa.right = b
                
            
    